Idea code name: `S1P10`

In [1]:
from keras.layers import Input, Dense, LSTM, PReLU, Reshape, Dropout, Activation, BatchNormalization
from keras.models import model_from_json, Model
import keras

from s1p10_model.resnet8_body import resnet8_body

from s1p10_training_utils import save_lstm, DataGenerator

import json
import time

import numpy as np

Using TensorFlow backend.


## Load weight of each angle class

In [2]:
with open('./s1p10_data/s1p10_classes_weight.json', 'r') as fp:
    classes_weight = json.load(fp)

## Load bins edge

In [3]:
bins_edge = np.load('./s1p10_data/s1p10_bins_edge.npy')

# Global Config

In [4]:
IMAGE_SHAPE = (200, 200, 1)
NUM_CLASSES = len(classes_weight)
NUM_PREDICT = 5
LSTM_NUM_HIDDEN_STATE = 64

# Model acrchitect

1 Image -> ResNet-8 -> LSTM_1 -> LSTM_2 (return state = True) (use this to sample prediction) -> Dense + Softmax 
-> 10 steering angle ID

## 1.2 Create Encoder
Encoder is body of ResNet-8 from `Drone-Net` since it's already trained to recognize road curve -> helpful spatial information


In [5]:
encoder = resnet8_body(IMAGE_SHAPE)
# load resnet8 weights
encoder.load_weights("./s1p10_model/named_resnet8_best_weights.h5", by_name=True)
for l in encoder.layers:
    l.trainable = False
encoder.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 200, 200, 1)  0                                            
__________________________________________________________________________________________________
conv_0 (Conv2D)                 (None, 100, 100, 32) 832         input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 49, 49, 32)   0           conv_0[0][0]                     
__________________________________________________________________________________________________
bn_1_a (BatchNorma

## 1.3 Create Decoder
Decoder is comprised of 2 LSTM & 2 Dense layers, last Dense layer is activated by Softmax function

In [6]:
LSTM_cell = LSTM(LSTM_NUM_HIDDEN_STATE, 
                 return_state=True, 
                 kernel_regularizer=keras.regularizers.l2(1e-2),
                 recurrent_regularizer=keras.regularizers.l2(1e-2),
                 bias_regularizer=keras.regularizers.l2(1e-2),
                 dropout=0.0, 
                 recurrent_dropout=0.0)  # use to sample angle

In [7]:
def dense_classifier(input_shape, num_classes, weight_regu_param):
    """
    Define classifier made of several Dense layers
    """
    X_in = Input(shape=input_shape)
    
    # 1st layer
    X = BatchNormalization()(X_in)
    
    X = Dense(64, 
              kernel_regularizer=keras.regularizers.l2(weight_regu_param),
              bias_regularizer=keras.regularizers.l2(weight_regu_param))(X_in)
    
    X = Activation('relu')(X)  # past use PReLU activation
    
    X = Dropout(0.5)(X)
    
    # 2nd layer
    X = BatchNormalization()(X)
    
    X = Dense(32, 
              kernel_regularizer=keras.regularizers.l2(weight_regu_param),
              bias_regularizer=keras.regularizers.l2(weight_regu_param))(X)
    
    X = Activation('relu')(X)  # past PReLU
    
    X = Dropout(0.5)(X)
    
    # 3rd layer 
    X = BatchNormalization()(X)
    
    X = Dense(8, 
              kernel_regularizer=keras.regularizers.l2(weight_regu_param),
              bias_regularizer=keras.regularizers.l2(weight_regu_param))(X)
    
    X_out = Activation('relu')(X)
    
#     X = Dropout(0.5)(X)
    
#     y = Dense(num_classes, activation='softmax', 
#               kernel_regularizer=keras.regularizers.l2(weight_regu_param), 
#               bias_regularizer=keras.regularizers.l2(weight_regu_param))(X)
    
    model = Model(inputs=[X_in], outputs=[X_out], name='classifier_body')
    return model
    

In [8]:
classifier = dense_classifier((LSTM_NUM_HIDDEN_STATE, ), NUM_CLASSES, weight_regu_param=0.01)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [9]:
classifier.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
activation_8 (Activation)    (None, 64)                0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 64)                256       
_________________________________________________________________
dense_2 (Dense)              (None, 32)                2080      
_________________________________________________________________
activation_9 (Activation)    (None, 32)                0         
__________

## 1.4 Create Model

In [10]:
def s1p10_model(input_shape, encoder, LSTM_cell, classifier, lstm_num_hidden_state, num_classes, num_labels):
    """
    Define see-1-predict-10 model
    
    Input:
        input_shape (tuple): shape of input image inputted to encoder
        encoder (keras.Model)
        LSTM_cell (keras.layers)
        classifier (keras.Model)
        
    Output:
        keras.Model
    """
    X_in = Input(shape=input_shape, name="image_in")
    
    # extract feature vector
    X_feature = encoder(X_in)
    X_feature = Reshape((1, -1))(X_feature)  # shape = (None, 1, 6272)
    
    # Initialize input to LSTM
    a0 = Input(shape=(LSTM_NUM_HIDDEN_STATE, ), name="a0")
    c0 = Input(shape=(LSTM_NUM_HIDDEN_STATE, ), name="c0")
    y0 = Input(shape=(num_classes, ), name="y0")  # will be concatenated with X_feature to make X
    
    a = a0
    c = c0
    y = y0
    
    outputs = []
    
    for i in range(num_labels):
        # concatenate y & X_feature
        y = Reshape((1, -1))(y)
        X = keras.layers.concatenate([X_feature, y], axis=-1)
        
        # propagate X through LSTM_cell
        a, _, c = LSTM_cell(X, initial_state=[a, c])
        
        # propagate hidden state "a" through classifier to get steering angle
        X = classifier(a)
        
        # output layer
        X = BatchNormalization()(X)
        
        X = Dropout(0.5)(X)
    
        X = Dense(num_classes, 
                  activation=None,
                  activity_regularizer=None,
                  kernel_regularizer=keras.regularizers.l2(1e-2), 
                  bias_regularizer=keras.regularizers.l2(1e-2))(X)
        
        y = Activation('softmax', name="head_%d" % i)(X)
        
        # store y
        outputs.append(y)
    
    model = Model(inputs=[X_in, a0, c0, y0], outputs=outputs)
    return model
        

In [11]:
model = s1p10_model(IMAGE_SHAPE, 
                    encoder, 
                    LSTM_cell, 
                    classifier, 
                    LSTM_NUM_HIDDEN_STATE, 
                    NUM_CLASSES, 
                    NUM_PREDICT)

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
image_in (InputLayer)           (None, 200, 200, 1)  0                                            
__________________________________________________________________________________________________
resnet8 (Model)                 (None, 6272)         309088      image_in[0][0]                   
__________________________________________________________________________________________________
y0 (InputLayer)                 (None, 114)          0                                            
__________________________________________________________________________________________________
reshape_1 (Reshape)             (None, 1, 6272)      0           resnet8[1][0]                    
__________________________________________________________________________________________________
reshape_2 

In [12]:
model.compile(optimizer='Adadelta', loss='categorical_crossentropy', metrics=['accuracy'])

# Training

In [13]:
batch_size = 80

gen_param = {'data_root_dir': "/home/user/Bureau/Dataset/udacity/", 
             'img_shape': IMAGE_SHAPE, 
             'num_class': NUM_CLASSES, 
             'num_prediction': NUM_PREDICT, 
             'bins_edge': bins_edge,
             'batch_size': batch_size, 
             'shuffle': True, 
             'lstm_dim_hidden_states': LSTM_NUM_HIDDEN_STATE,
             'flip_prob': 0.5}

train_gen = DataGenerator("./s1p10_data/s1p10_CH2_002_output_training.csv", **gen_param)
val_gen = DataGenerator("./s1p10_data/s1p10_CH2_002_output_validation.csv", **gen_param)

In [14]:
time_str = time.strftime("%Y_%m_%d_%H_%M")
log_dir = './s1p10_logs/' + time_str
tb_callback = keras.callbacks.TensorBoard(log_dir=log_dir,  
                                          batch_size=batch_size, 
                                          update_freq='epoch')

In [15]:
model.fit_generator(train_gen,
                    class_weight=classes_weight,
                    epochs=7,
                    validation_data=val_gen,
                    initial_epoch=0,
                    callbacks=[tb_callback])

Instructions for updating:
Use tf.cast instead.
Epoch 1/7
227/227 [==============================] - 114s 502ms/step - loss: 25.5821 - head_0_loss: 4.5049 - head_1_loss: 4.5112 - head_2_loss: 4.5173 - head_3_loss: 4.4775 - head_4_loss: 4.5147 - head_0_acc: 0.0905 - head_1_acc: 0.0597 - head_2_acc: 0.0971 - head_3_acc: 0.1077 - head_4_acc: 0.0948 - val_loss: 22.3424 - val_head_0_loss: 4.2242 - val_head_1_loss: 3.9844 - val_head_2_loss: 4.1271 - val_head_3_loss: 4.2539 - val_head_4_loss: 3.9487 - val_head_0_acc: 0.1976 - val_head_1_acc: 0.1063 - val_head_2_acc: 0.2043 - val_head_3_acc: 0.1713 - val_head_4_acc: 0.2263
Epoch 2/7
227/227 [==============================] - 107s 470ms/step - loss: 19.8549 - head_0_loss: 3.6231 - head_1_loss: 3.5806 - head_2_loss: 3.7300 - head_3_loss: 3.6258 - head_4_loss: 3.6600 - head_0_acc: 0.2090 - head_1_acc: 0.1941 - head_2_acc: 0.2003 - head_3_acc: 0.2101 - head_4_acc: 0.2047 - val_loss: 17.8099 - val_head_0_loss: 3.2701 - val_head_1_loss: 3.2146 - val

# Save weights

In [16]:
# save shared_lstm
save_lstm(LSTM_cell, time_str, log_dir)

# save classifier
classifier.save_weights(log_dir + "/classifier_%s.h5" % time_str)

In [17]:
time_str

'2019_05_28_11_15'